# JSON layout data

In [1]:
import json
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#get the layout-data from the json file

jsonfile = "/content/drive/MyDrive/MM Project/Data/layout.json"
with open(jsonfile, 'r') as json_file:
    data = json.load(json_file)
    #for i in data['beacons']:
    #    print(i)
    #for i in data['regions']:
    #    print(i)

    data_regions = pd.DataFrame.from_dict(data['regions'])
    data_beacons = pd.DataFrame.from_dict(data['beacons'])


In [3]:
data_merged = data_beacons.merge(data_regions, left_on='region_uuid', right_on='uuid')

#print(data_merged.head())
#print(data_beacons.shape)
#print(data_regions.shape)
#print(data_merged.shape)

data_merged.rename(columns={'id_x': 'beacon_id', 'id_y': 'region_id', 'name': 'region_name', 'position_x': 'b_pos_x', 'position_y': 'b_pos_y', \
'position_top_left_x': 'rpos_top_left_x', 'position_top_left_y': 'rpos_top_left_y', \
'position_bottom_right_x': 'rpos_bottom_right_x', 'position_bottom_right_y': 'rpos_bottom_right_y'}, inplace=True)
data_merged.drop(['uuid_x', 'comment', 'region_uuid', 'uuid_y', 'layout_uuid_y', 'layout_uuid_x', 'exclude_from_eval', 'special_activities', 'to_be_deleted', 'label_uuid'], axis=1, inplace=True)
data_merged['region_beacon'] = data_merged['region_id'].astype(str) + "_" + data_merged['beacon_id'].astype(str)

#print(data_merged.head())

In [4]:
expected_flow = {'flow_region_name': ['pre-checkin', 'waiting checkin', 'checkin main', 'waiting', 'doctor table', 'vaccination', 'waiting II', 'checkout', 'waiting III'], \
'flow_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],\
'cor_region': [[1, 2, 5], 90, [3, 4], 91, [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 92, [30, 31, 32, 33, 34 ,35, 36, 37 ,38, 39, 40, 41], 93]}

expected_regions = pd.DataFrame.from_dict(data=expected_flow)
#print(expected_regions)

exp_long = expected_regions.explode('cor_region')
print(exp_long.shape)

(41, 3)


In [5]:
# adjust the region flow -> create a region/flow 0 for "Pause/Not in use"

adj_expected_flow = {'flow_region_name': ['Not in use', 'pre-checkin', 'waiting checkin', 'checkin main', 'waiting', 'doctor table', 'vaccination', 'waiting II', 'checkout', 'waiting III'], \
'flow_id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],\
'cor_region': [1, [2, 5], 90, [3, 4], 91, [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 92, [30, 31, 32, 33, 34 ,35, 36, 37 ,38, 39, 40, 41], 93]}

adj_expected_regions = pd.DataFrame.from_dict(data=adj_expected_flow)
#print(expected_regions)

adj_exp_long = adj_expected_regions.explode('cor_region')
print(exp_long.shape)

(41, 3)


In [ ]:
### JUST TESTING, DONT RUN
organisation_structure_dataframe = {'region_name': ['Not in use', 'pre-checkin', 'waiting checkin', 'checkin main', 'waiting', 'doctor table', 'vaccination', 'waiting II', 'checkout', 'waiting III'], \
'flow_id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],\
'beacons': ["Beacon_271","Beacon_257","Beacon_256","Beacon_255","Beacon_254","Beacon_253"], [], ["Beacon_206","Beacon_207","Beacon_208","Beacon_209","Beacon_210"], [], \
["Beacon_211","Beacon_212","Beacon_213","Beacon_214","Beacon_215","Beacon_216","Beacon_217","Beacon_218","Beacon_219","Beacon_220"], \
["Beacon_221","Beacon_222","Beacon_223","Beacon_224","Beacon_225","Beacon_226","Beacon_227","Beacon_228","Beacon_229",\
 "Beacon_230","Beacon_231","Beacon_232","Beacon_233","Beacon_234","Beacon_235","Beacon_236","Beacon_237","Beacon_238","Beacon_239","Beacon_240"], \
 [], ["Beacon_241","Beacon_242","Beacon_243","Beacon_244","Beacon_245","Beacon_246","Beacon_247","Beacon_248","Beacon_249","Beacon_250","Beacon_251","Beacon_272"], []}


adj_expected_regions = pd.DataFrame.from_dict(data=adj_expected_flow)
#print(expected_regions)

SyntaxError: ignored

In [6]:
# CORRECTION: used the adjusted flow now

data_merged_new = data_merged.merge(adj_exp_long, left_on='region_id', right_on='cor_region')
data_merged_new['flow_beacon'] = data_merged_new['flow_id'].astype(str) + "_" + data_merged_new['beacon_id'].astype(str)
data_merged_new.drop('cor_region', axis=1, inplace=True)

print(data_merged_new)

    beacon_id   b_pos_x   b_pos_y  ... flow_region_name flow_id  flow_beacon
0         226  0.727975  0.644562  ...      vaccination       6        6_226
1         225  0.727975  0.687666  ...      vaccination       6        6_225
2         215  0.660764  0.545243  ...     doctor table       5        5_215
3         245  0.857265  0.525199  ...         checkout       8        8_245
4         260  0.354315  0.981687  ...       Not in use       0        0_260
..        ...       ...       ...  ...              ...     ...          ...
56        244  0.857265  0.494032  ...         checkout       8        8_244
57        219  0.665982  0.307433  ...     doctor table       5        5_219
58        218  0.667114  0.368246  ...     doctor table       5        5_218
59        220  0.664094  0.243120  ...     doctor table       5        5_220
60        212  0.658793  0.716634  ...     doctor table       5        5_212

[61 rows x 13 columns]


Create a df with the flow regions and the corresponding beacons

In [7]:
multi_col_flow = data_merged_new.loc[:, ['beacon_id', 'flow_id']] #only the beacon and flow region ids are relevant here
multi_col_flow.sort_values(by = ['flow_id', 'beacon_id'], inplace=True)
multi_col_flow = multi_col_flow[['flow_id', 'beacon_id']] #change order of the columns
print(multi_col_flow)

    flow_id  beacon_id
9         0        201
6         0        202
5         0        203
8         0        258
7         0        259
..      ...        ...
53        8        248
43        8        249
26        8        250
51        8        251
25        8        272

[61 rows x 2 columns]


In [8]:
#############HERE#############

# get tuples from region+beacon in order to use pd.MultiIndex.from_tuples

multi_col_flow_records = multi_col_flow.to_records(index=False)
multi_col_flow_tuple = list(multi_col_flow_records)
print(multi_col_flow_tuple)

# sort the tuple by beacon ids
multi_col_flow_tuple.sort(key=lambda tup: tup[1]) 
print(multi_col_flow_tuple)

[(0, 201), (0, 202), (0, 203), (0, 258), (0, 259), (0, 260), (1, 253), (1, 254), (1, 255), (1, 256), (1, 257), (1, 271), (3, 204), (3, 205), (3, 206), (3, 207), (3, 208), (3, 209), (3, 210), (5, 211), (5, 212), (5, 213), (5, 214), (5, 215), (5, 216), (5, 217), (5, 218), (5, 219), (5, 220), (6, 221), (6, 222), (6, 223), (6, 224), (6, 225), (6, 226), (6, 227), (6, 228), (6, 229), (6, 230), (6, 231), (6, 232), (6, 233), (6, 234), (6, 235), (6, 236), (6, 237), (6, 238), (6, 239), (6, 240), (8, 241), (8, 242), (8, 243), (8, 244), (8, 245), (8, 246), (8, 247), (8, 248), (8, 249), (8, 250), (8, 251), (8, 272)]
[(0, 201), (0, 202), (0, 203), (3, 204), (3, 205), (3, 206), (3, 207), (3, 208), (3, 209), (3, 210), (5, 211), (5, 212), (5, 213), (5, 214), (5, 215), (5, 216), (5, 217), (5, 218), (5, 219), (5, 220), (6, 221), (6, 222), (6, 223), (6, 224), (6, 225), (6, 226), (6, 227), (6, 228), (6, 229), (6, 230), (6, 231), (6, 232), (6, 233), (6, 234), (6, 235), (6, 236), (6, 237), (6, 238), (6, 239)

# Beacon data

In [9]:
import pickle #to extract the pickle file
import matplotlib.pyplot as plt
import numpy as np #to work with numpy arrays like rssi_arr
import pandas as pd
from pandas.core.indexes.base import Index #to create dataframes
import seaborn as sns #make visualisation with df´s

In [10]:
def extract_rssi_to_df(folderpath):
    """ Takes path from pickle file and create a df with a timeline and the rssi_arr values """
    #create a dataframe df from the picke.file
    df_file = pickle.load(open( folderpath, "rb" ) )

    #extract rssi_arr key to numpy.array
    rssi_arr=df_file["rssi_arr"]
    
    #set -inf equal to 0 (datacleaning)
    rssi_arr[rssi_arr == -np.Inf] = np.nan

    #figure out the shape from rssi_arr array for the timeline
    dim=np.shape(rssi_arr)

    #create a timeline and add it to the rssi_arr array
    timeline=np.linspace(0.1, (dim[0])*0.1, dim[0]).reshape(dim[0],1) #problem that it changes from 0091 file to others why?
    mod_rssi_arr = np.append(rssi_arr, timeline, axis = 1)

    #create the colum_names for df_rssi_arr (dataframe)
    ##colum_names=[]
    ##for i in range(1,dim[1]+1):
        ##value="Beacon_" + str(200+i)
        ##colum_names.append(value)
    ##colum_names.append("timeline[s]")

    # beacon id as column
    df_beacon_id = df_file['beacon_uuids']
    df_beacon_id = np.append(df_beacon_id, "timeline[s]")

    #create df_rssi_arr 
    df_rssi_arr= pd.DataFrame(data=mod_rssi_arr,columns=df_beacon_id)
    df=df_rssi_arr.set_index("timeline[s]")
    return df

In [11]:
Path1="/content/drive/MyDrive/Project: MotionMiners/Data/MMTS_00091_1620884436927.bin_beacon_data.pickle"          #Pfad anpassen falls andere damit arbeiten
MMTS_00091_1620884436927=extract_rssi_to_df(Path1)
print("MMTS_00091_1620884436927:")
print(MMTS_00091_1620884436927)

MMTS_00091_1620884436927:
              201   202   203  204  205  206  ...   270  271  272   273   274   275
timeline[s]                                   ...                                  
0.1         -74.0   NaN -50.0  NaN  NaN  NaN  ... -61.0  NaN  NaN -65.0 -61.0 -62.0
0.2         -74.0 -47.0 -50.0  NaN  NaN  NaN  ... -61.0  NaN  NaN -65.0 -61.0 -62.0
0.3         -74.0 -47.0 -50.0  NaN  NaN  NaN  ... -61.0  NaN  NaN -65.0 -61.0 -62.0
0.4         -74.0 -47.0 -50.0  NaN  NaN  NaN  ... -61.0  NaN  NaN -65.0 -61.0 -62.0
0.5         -48.0 -47.0 -50.0  NaN  NaN  NaN  ... -61.0  NaN  NaN -65.0 -60.0 -60.0
...           ...   ...   ...  ...  ...  ...  ...   ...  ...  ...   ...   ...   ...
42119.2     -61.0 -67.0 -64.0  NaN  NaN  NaN  ... -78.0  NaN  NaN -80.0 -76.0 -72.0
42119.3     -62.0 -64.0 -62.0  NaN  NaN  NaN  ... -73.0  NaN  NaN -80.0 -72.0 -72.0
42119.4     -62.0 -64.0 -62.0  NaN  NaN  NaN  ... -73.0  NaN  NaN -80.0 -72.0 -72.0
42119.5     -62.0 -64.0 -62.0  NaN  NaN  NaN  ... 

# Dataframe with multi column

- compare the beacon ids
- delete unimportant columns
- use pd.MultiIndex.from_tuples

In [12]:
# find all beacon ids that were not used
 
idint = MMTS_00091_1620884436927.columns.values.astype(int) # need to change column label type to integer; idint -> (beacon) id int(eger)
not_used_ids = []

for i in idint:
  if i not in list(multi_col_flow['beacon_id']):
    not_used_ids.append(i)

print(not_used_ids)

[261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 273, 274, 275]


In [13]:
#delete the columns of the beacons that were not used

MMTS_00091_1620884436927.columns = MMTS_00091_1620884436927.columns.map(int) # need to change column label type to integer 
MMTS_00091_1620884436927 = MMTS_00091_1620884436927.drop(not_used_ids, axis=1)

print(MMTS_00091_1620884436927.columns.values)

[201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 253 254 255
 256 257 258 259 260 271 272]


In [14]:
# Creating the MultiIndex
MMTS_00091_1620884436927.columns = pd.MultiIndex.from_tuples(multi_col_flow_tuple, names =('Flow', 'Beacon'))
  
# Print the MultiIndex
print(MMTS_00091_1620884436927)

Flow            0               3              ...   1         0               1   8
Beacon        201   202   203 204 205 206 207  ... 256 257   258   259   260 271 272
timeline[s]                                    ...                                  
0.1         -74.0   NaN -50.0 NaN NaN NaN NaN  ... NaN NaN -57.0 -71.0 -73.0 NaN NaN
0.2         -74.0 -47.0 -50.0 NaN NaN NaN NaN  ... NaN NaN -57.0 -71.0 -73.0 NaN NaN
0.3         -74.0 -47.0 -50.0 NaN NaN NaN NaN  ... NaN NaN -57.0 -71.0 -73.0 NaN NaN
0.4         -74.0 -47.0 -50.0 NaN NaN NaN NaN  ... NaN NaN -57.0 -71.0 -73.0 NaN NaN
0.5         -48.0 -47.0 -50.0 NaN NaN NaN NaN  ... NaN NaN -57.0 -57.0 -73.0 NaN NaN
...           ...   ...   ...  ..  ..  ..  ..  ...  ..  ..   ...   ...   ...  ..  ..
42119.2     -61.0 -67.0 -64.0 NaN NaN NaN NaN  ... NaN NaN -62.0 -59.0 -50.0 NaN NaN
42119.3     -62.0 -64.0 -62.0 NaN NaN NaN NaN  ... NaN NaN -65.0 -58.0 -53.0 NaN NaN
42119.4     -62.0 -64.0 -62.0 NaN NaN NaN NaN  ... NaN NaN -65.0 

In [15]:
#############HERE#############

MMTS_00091_1620884436927 = MMTS_00091_1620884436927[np.sort(MMTS_00091_1620884436927.columns)]

print(MMTS_00091_1620884436927[np.sort(MMTS_00091_1620884436927.columns)])

Flow            0                                 1  ...   8                        
Beacon        201   202   203   258   259   260 253  ... 246 247 248 249 250 251 272
timeline[s]                                          ...                            
0.1         -74.0   NaN -50.0 -57.0 -71.0 -73.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
0.2         -74.0 -47.0 -50.0 -57.0 -71.0 -73.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
0.3         -74.0 -47.0 -50.0 -57.0 -71.0 -73.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
0.4         -74.0 -47.0 -50.0 -57.0 -71.0 -73.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
0.5         -48.0 -47.0 -50.0 -57.0 -57.0 -73.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
...           ...   ...   ...   ...   ...   ...  ..  ...  ..  ..  ..  ..  ..  ..  ..
42119.2     -61.0 -67.0 -64.0 -62.0 -59.0 -50.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
42119.3     -62.0 -64.0 -62.0 -65.0 -58.0 -53.0 NaN  ... NaN NaN NaN NaN NaN NaN NaN
42119.4     -62.0 -64.0 -62.0 -65.0 -58.0 -53.0 NaN  ... NaN NaN 

In [ ]:
print(MMTS_00091_1620884436927.loc[:,1].max())

Beacon
253   -56.0
254   -55.0
255   -46.0
256   -52.0
257   -49.0
271   -60.0
dtype: float64


In [ ]:
print(MMTS_00091_1620884436927.loc[:,1].min())

Beacon
253   -91.0
254   -91.0
255   -90.0
256   -90.0
257   -91.0
271   -92.0
dtype: float64


# Analysis to find out how many persons used this tracer

In [ ]:
print(print(MMTS_00091_1620884436927.iloc[1,:].idxmax(axis=1)))

(1, 202)
None


In [ ]:
#for index, value in MMTS_00091_1620884436927[1].iterrows():
#  print(index, value)

# Saving df as separate file

In [ ]:
# csv data
MMTS_00091_1620884436927.to_csv(r'/content/drive/MyDrive/MM Project/Data/tracer_91.csv')

In [ ]:
#############HERE#############
# pickle data

MMTS_00091_1620884436927.to_pickle('/content/drive/MyDrive/MM Project/Data/tracer_91.pickle')

TO DO

In [ ]:
# saving with a function

def save_dfs_cvs(df_name):
  df_name.to_csv(r'/content/drive/MyDrive/PM Project (PRIVATE)/Data/') #add name for data file 

# Graph

In [ ]:
## funktioniert leider nach meinen Änderungen nicht mehr?

y="272"
MMTS_00091_1620884436927.loc[MMTS_00091_1620884436927[y] == 0,y]=np.nan
print(MMTS_00091_1620884436927[y])
sns.relplot(x="timeline[s]",y=y,data=MMTS_00091_1620884436927,kind="scatter")
plt.show()

KeyError: ignored

# Other stuff
- needed to check an error

In [ ]:
# check where error is

df_file = pickle.load(open( Path1, "rb" ) )
rssi_arr=df_file["rssi_arr"]
rssi_arr[rssi_arr == -np.Inf] = 0
dim=np.shape(rssi_arr)
timeline=np.linspace(0.1, (dim[0])*0.1, dim[0]).reshape(dim[0],1)
mod_rssi_arr = np.append(rssi_arr, timeline, axis = 1)

df_beacon_id = df_file['beacon_uuids']
df_beacon_id = np.append(df_beacon_id, "timeline[s]")

df_rssi_arr= pd.DataFrame(data=mod_rssi_arr,columns=df_beacon_id)

# working

In [ ]:
for index, i in multi_col_flow['beacon_id'].iteritems():
  #print(i)
  if i in list(MMTS_00091_1620884436927.columns.values.astype(int)):
    print(i)
  #for j in list(MMTS_00091_1620884436927.columns.values):
    #print(j)
   # if i == j:
    #  print(j)

201
202
203
253
254
255
256
257
258
259
260
271
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
272


In [ ]:
print(multi_col_flow['beacon_id'])
print(MMTS_00091_1620884436927.columns.values)
print(list(multi_col_flow['beacon_id']))

9     201
6     202
5     203
15    253
12    254
     ... 
53    248
43    249
26    250
51    251
25    272
Name: beacon_id, Length: 61, dtype: int64
['201' '202' '203' '204' '205' '206' '207' '208' '209' '210' '211' '212'
 '213' '214' '215' '216' '217' '218' '219' '220' '221' '222' '223' '224'
 '225' '226' '227' '228' '229' '230' '231' '232' '233' '234' '235' '236'
 '237' '238' '239' '240' '241' '242' '243' '244' '245' '246' '247' '248'
 '249' '250' '251' '253' '254' '255' '256' '257' '258' '259' '260' '261'
 '262' '263' '264' '265' '266' '267' '268' '269' '270' '271' '272' '273'
 '274' '275']
[201, 202, 203, 253, 254, 255, 256, 257, 258, 259, 260, 271, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 272]
